In [ ]:
import sys
import os
from pathlib import Path

# srcをパスに追加
# ノートブックの場所: /portfolio/01_fraud_neuromorphic/notebooks/
# srcの場所: /portfolio/01_fraud_neuromorphic/src/
notebook_dir = Path.cwd()

# プロジェクトルートディレクトリまたはnotebooksディレクトリにいるか確認
if (notebook_dir / 'portfolio' / '01_fraud_neuromorphic' / 'src').exists():
    # プロジェクトルート（Project-Neuromorfico-X）にいる
    src_path = notebook_dir / 'portfolio' / '01_fraud_neuromorphic' / 'src'
elif (notebook_dir.parent / 'src').exists():
    # notebooks/にいる
    src_path = notebook_dir.parent / 'src'
elif (notebook_dir / 'src').exists():
    # 01_fraud_neuromorphic/にいる
    src_path = notebook_dir / 'src'
else:
    src_path = None

if src_path and src_path.exists():
    if str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
        print(f"✅ srcディレクトリを追加: {src_path}")
else:
    print(f"⚠️ srcディレクトリが見つかりません！")
    print(f"  ノートブックディレクトリ: {notebook_dir}")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
from tqdm.auto import tqdm
import brian2

# Brian2をnumpyを使用するように設定（C++コンパイルエラーとSymPy/Cythonの問題を回避）
brian2.prefs.codegen.target = "numpy"

# プロジェクトモジュール
try:
    from main import FraudDetectionPipeline, generate_synthetic_transactions
    from encoders import RateEncoder, timeralEncoder, PopulationEncoder, TransactionEncoder
    from models_snn import FraudSNN, demonstrate_lif_neuron  # type: ignore[attr-defined]
    print("✅ プロジェクトモジュールのインポート完了！")
except ImportError as e:
    print(f"⚠️ モジュールのインポートエラー: {e}")
    print(f"  sys.path: {sys.path[:3]}")

# 視覚化設定
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 2. 合成データ生成

現実的なパターンを持つ銀行取引の合成データセットを作成しましょう。

In [ ]:
# 1000件の取引を生成（不正率5%）
print("合成取引を生成しています...")
df = generate_synthetic_transactions(n=1000, fraud_ratio=0.05)

print(f"\n📊 データセット生成完了:")
print(f"  総取引数: {len(df)}")
print(f"  正当な取引: {np.sum(df['is_fraud'] == 0)}")
print(f"  不正取引: {np.sum(df['is_fraud'] == 1)}")
print(f"  不正率: {df['is_fraud'].mean():.2%}")

# 最初の行を表示
df.head(10)

In [ ]:
# 値の分布を視覚化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# クラス別の値の分布
df[df['is_fraud'] == 0]['amount'].hist(bins=50, alpha=0.7, label='正当', ax=axes[0], color='green')
df[df['is_fraud'] == 1]['amount'].hist(bins=30, alpha=0.7, label='不正', ax=axes[0], color='red')
axes[0].set_xlabel('取引額（$）')
axes[0].set_ylabel('頻度')
axes[0].set_title('クラス別値の分布')
axes[0].legend()

# クラス別の日次頻度
df.boxplot(column='daily_frequency', by='is_fraud', ax=axes[1])
axes[1].set_xlabel('クラス（0=正当、1=不正）')
axes[1].set_ylabel('日次頻度')
axes[1].set_title('クラス別取引頻度')
plt.suptitle('')

plt.tight_layout()
plt.show()

print("\n📊 観察されたパターン:")
print("  - 不正取引は高額になる傾向がある")
print("  - 不正取引は取引頻度が高い")

## 3. スパイクエンコーディング

取引特徴を時間的スパイクに変換する方法を実演します。

In [ ]:
# レートエンコーディングの例
print("=== レートエンコーディング ===")
print("連続値をスパイク頻度としてエンコード\n")

rate_encoder = RateEncoder(min_rate=1, max_rate=100, duration=0.1)

# 異なる値でテスト
test_amounts = [100, 500, 1000, 5000, 10000]

fig, axes = plt.subplots(len(test_amounts), 1, figsize=(12, 10))

for idx, amount in enumerate(test_amounts):
    spike_times = rate_encoder.encode(amount, min_val=0, max_val=10000)
    
    # 視覚化
    if spike_times:
        axes[idx].eventplot([spike_times], linewidths=2, colors='blue')
        axes[idx].set_xlim(0, 0.1)
        axes[idx].set_ylim(0.5, 1.5)
        axes[idx].set_ylabel(f'${amount}')
        axes[idx].set_title(f'値: ${amount} → {len(spike_times)} スパイク')
        axes[idx].grid(True, alpha=0.3)
    
axes[-1].set_xlabel('時間（秒）')
plt.suptitle('レートエンコーディング：値 → スパイク頻度', fontsize=14, y=1.0)
plt.tight_layout()
plt.show()

print("\n💡 注：高い値はより多くのスパイクを生成（高い頻度）")

In [ ]:
# ポピュレーションエンコーディングの例（地理的位置）
print("=== ポピュレーションエンコーディング ===")
print("受容野を持つ複数のニューロンを使用して値をエンコード\n")

pop_encoder = PopulationEncoder(n_neurons=20, min_val=-1, max_val=1, sigma=0.15)

# 異なる位置でテスト
test_locations = [-0.8, -0.3, 0.0, 0.4, 0.9]

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# プロット1: ニューロン活性化
for loc in test_locations:
    activations = np.exp(-((pop_encoder.centers - loc) ** 2) / (2 * pop_encoder.sigma ** 2))
    axes[0].plot(pop_encoder.centers, activations, marker='o', label=f'位置 = {loc:.1f}', alpha=0.7)

axes[0].set_xlabel('ニューロン中心')
axes[0].set_ylabel('活性化')
axes[0].set_title('位置によるニューロン集団の活性化')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# プロット2: スパイクのラスタープロット
spike_data = []
for idx, loc in enumerate(test_locations):
    encoding = pop_encoder.encode(loc, duration=0.1)
    if len(encoding.spike_times) > 0:
        for t, n in zip(encoding.spike_times, encoding.neuron_indices):
            spike_data.append([t, n + idx * 25]) # 視覚化のためのオフセット

if spike_data:
    spike_array = np.array(spike_data)
    axes[1].scatter(spike_array[:, 0], spike_array[:, 1], marker='|', s=100, alpha=0.6)

axes[1].set_xlabel('時間（秒）')
axes[1].set_ylabel('ニューロン + オフセット')
axes[1].set_title('ニューロン集団によって生成されたスパイク')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 注：各位置は異なるニューロングループを活性化")

## 4. SNNアーキテクチャ

スパイキングニューラルネットワークアーキテクチャを視覚化して理解します。

In [ ]:
# 個別LIFニューロンのデモンストレーション
print("=== 漏れ積分発火ニューロン ===")
print("LIFニューロンの挙動のデモンストレーション\n")

lif_data = demonstrate_lif_neuron()  # type: ignore[name-defined]

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# プロット1: 入力電流
axes[0].plot(lif_data['time'], lif_data['input'], color='blue', linewidth=2)
axes[0].set_ylabel('入力電流（I）')
axes[0].set_title('入力刺激（ステップ電流）')
axes[0].grid(True, alpha=0.3)

# プロット2: 膜電位とスパイク
axes[1].plot(lif_data['time'], lif_data['voltage'], color='green', linewidth=2, label='膜電位')
axes[1].axhline(-50, color='red', linestyle='--', label='閾値（-50mV）', alpha=0.7)
axes[1].axhline(-70, color='gray', linestyle='--', label='静止（-70mV）', alpha=0.5)

# スパイクをマーク
for spike_time in lif_data['spikes']:
    axes[1].axvline(spike_time, color='red', alpha=0.3, linewidth=1)

axes[1].set_xlabel('時間（ms）')
axes[1].set_ylabel('電圧（mV）')
axes[1].set_title(f'膜電位（合計 {len(lif_data["spikes"])} スパイク）')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 分析:")
print(f"  - 検出されたスパイク: {len(lif_data['spikes'])}")
print(f"  - 平均周波数: {len(lif_data['spikes']) / (lif_data['time'][-1] / 1000):.1f} Hz")
print(f"  - 平均ISI: {np.mean(np.diff(lif_data['spikes'])):.2f} ms" if len(lif_data['spikes']) > 1 else "")

In [ ]:
# SNNアーキテクチャ統計
print("=== SNNアーキテクチャ ===")

snn = FraudSNN(input_size=256, hidden_sizes=[128, 64], output_size=2)
stats = snn.get_network_stats()  # type: ignore[attr-defined]

print(f"\n📊 ネットワーク構造:")
print(f"  入力層: {stats['layers']['input']} ニューロン")
print(f"  隠れ層1: {stats['layers']['hidden'][0]} ニューロン（LIF）")
print(f"  隠れ層2: {stats['layers']['hidden'][1]} ニューロン（LIF）")
print(f"  出力層: {stats['layers']['output']} ニューロン")
print(f"\n  総ニューロン数: {stats['total_neurons']}")
print(f"  総シナプス数: {stats['total_synapses']}")

print(f"\n⚖️ シナプス重み:")
print(f"  平均: {stats['weights']['mean']:.4f}")
print(f"  標準偏差: {stats['weights']['std']:.4f}")
print(f"  最小: {stats['weights']['min']:.4f}")
print(f"  最大: {stats['weights']['max']:.4f}")

# アーキテクチャを視覚化
layer_sizes = [256, 128, 64, 2]
layer_names = ['入力\n(256)', '隠れ層1\n(128)', '隠れ層2\n(64)', '出力\n(2)']

fig, ax = plt.subplots(figsize=(14, 6))

# 層を描画
x_positions = np.linspace(0, 10, len(layer_sizes))
max_size = max(layer_sizes)

for i, (size, name, x) in enumerate(zip(layer_sizes, layer_names, x_positions)):
    y_positions = np.linspace(0, max_size, size)
    
    # 大きな層の視覚化を制限
    display_neurons = min(size, 20)
    y_display = np.linspace(0, max_size, display_neurons)
    
    ax.scatter([x] * display_neurons, y_display, s=100, alpha=0.7, 
               color=f'C{i}', label=name, zorder=3)
    
    # 次の層と接続
    if i < len(layer_sizes) - 1:
        next_x = x_positions[i + 1]
        next_size = min(layer_sizes[i + 1], 20)
        next_y = np.linspace(0, max_size, next_size)
        
        # いくつかの接続を描画（サンプル）
        for y1 in y_display[::3]:
            for y2 in next_y[::3]:
                ax.plot([x, next_x], [y1, y2], 'k-', alpha=0.05, linewidth=0.5, zorder=1)

ax.set_xlim(-1, 11)
ax.set_ylim(-20, max_size + 20)
ax.set_xticks(x_positions)
ax.set_xticklabels(layer_names)
ax.set_yticks([])
ax.set_title('不正検出のためのスパイキングニューラルネットワークアーキテクチャ', fontsize=14)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
plt.show()

## 5. 完全なパイプライン

エンドツーエンドのパイプラインを実行：トレーニングと評価。

In [ ]:
# パイプラインを初期化
print("不正検出パイプラインを初期化しています...")
pipeline = FraudDetectionPipeline()

# 訓練/テスト分割 - 高速デモのために小さなデータを使用
train_size = int(0.8 * len(df))
train_data = df[:train_size].copy()
test_data = df[train_size:].copy()

# デモのため、訓練の小さなサブセットのみを使用
# numpyモード（C++コンパイルなし）での高速実行のために20サンプルに削減
train_subset_size = min(20, len(train_data))
train_subset = train_data.sample(n=train_subset_size, random_state=42)

print(f"\nデータの分割:")
print(f" 訓練（デモ用サブセット）: {len(train_subset)} 取引")
print(f" テスト: {len(test_data)} 取引")
print(f"\n注：高速デモのために削減されたサブセットを使用")

print("\n⏳ STDPによるトレーニング開始...")
print("（高速デモのために少数のエポックと削減されたデータを使用）\n")

# デモのために大幅に削減
epochs = 2
print(f" トレーニング: {epochs} エポック、{len(train_subset)} 取引")

# 高速トレーニング
start_time = time.time()
print("トレーニング中...")
pipeline.train(train_subset, epochs=epochs)
training_time = time.time() - start_time

print(f"\nトレーニングが{training_time:.1f}秒で完了")
print(f" エポックあたりの平均時間: {training_time/epochs:.2f}秒")
print(f" レート: {len(train_subset) * epochs / training_time:.1f} 取引/秒")

In [ ]:
# テストセットで評価
print("テストセットでモデルを評価しています...")
print(f"合計 {len(test_data)} 取引\n")

# 推定時間で評価
start_eval = time.time()
metrics = pipeline.evaluate(test_data)
eval_time = time.time() - start_eval

print(f"\n評価が{eval_time:.2f}秒で完了")
print(f" 速度: {len(test_data)/eval_time:.1f} 取引/秒")


In [ ]:
# 混同行列を視覚化
from sklearn.metrics import confusion_matrix
import numpy as np

# numpy配列に明示的に変換
y_true = test_data['is_fraud'].to_numpy()
y_pred = []

print("混同行列の予測を生成しています...")
for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="予測", unit="txn"):
    result = pipeline.predict(row.to_dict())
    y_pred.append(int(result['is_fraud']))

# y_predをnumpy配列に変換
y_pred = np.array(y_pred, dtype=int)

cm = confusion_matrix(y_true, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['正当', '不正'],
            yticklabels=['正当', '不正'],
            cbar_kws={'label': 'カウント'})
ax.set_xlabel('予測')
ax.set_ylabel('実際')
ax.set_title('混同行列 - ニューロモーフィック不正検出')
plt.tight_layout()
plt.show()

print(f"\n精度: {metrics['accuracy']:.2%}")
print(f"  適合率: {metrics['precision']:.2%}")
print(f"  再現率: {metrics['recall']:.2%}")
print(f"  F1スコア: {metrics['f1_score']:.2%}")

## 6. 個別予測の例

特定の取引でテストします。

In [ ]:
# 例1: 典型的な正当な取引
print("=" * 60)
print("例1: 正当な取引")
print("=" * 60)

legitimate_txn = {
 'id': 'demo_001',
 'amount': 85.50,
 'timestamp': time.time(),
 'merchant_category': 'groceries',
 'location': (-23.5505, -46.6333), # サンパウロ
 'device_id': 'device_regular_001',
 'daily_frequency': 3
}

result = pipeline.predict(legitimate_txn)

print(f"\n取引:")
print(f" 金額: ${legitimate_txn['amount']:.2f}")
print(f" カテゴリ: {legitimate_txn['merchant_category']}")
print(f" 位置: サンパウロ")

print(f"\n分析結果:")
print(f" 不正検出: {'はい' if result['is_fraud'] else 'いいえ'}")
print(f" 信頼度: {result['confidence']:.2%}")
print(f" 正当スコア: {result['legitimate_score']:.2f} Hz")
print(f" 不正スコア: {result['fraud_score']:.2f} Hz")
print(f" レイテンシ: {result['latency_ms']:.2f}ms")
print(f" 生成されたスパイク: {result['n_spikes_generated']}")

In [ ]:
# 例2: 疑わしい取引（不正の高い確率）
print("=" * 60)
print("例2: 疑わしい取引")
print("=" * 60)

suspicious_txn = {
 'id': 'demo_002',
 'amount': 8500.00, # 高額
 'timestamp': time.time(),
 'merchant_category': 'electronics',
 'location': (51.5074, -0.1278), # ロンドン（異常な位置）
 'device_id': 'device_new_unknown', # 新しいデバイス
 'daily_frequency': 25 # 異常な頻度
}

result = pipeline.predict(suspicious_txn)

print(f"\n取引:")
print(f" 金額: ${suspicious_txn['amount']:.2f}")
print(f" カテゴリ: {suspicious_txn['merchant_category']}")
print(f" 位置: ロンドン（異常）")
print(f" デバイス: 新規/不明")

print(f"\n分析結果:")
print(f" 不正検出: {'はい' if result['is_fraud'] else 'いいえ'}")
print(f" 信頼度: {result['confidence']:.2%}")
print(f" 正当スコア: {result['legitimate_score']:.2f} Hz")
print(f" 不正スコア: {result['fraud_score']:.2f} Hz")
print(f" レイテンシ: {result['latency_ms']:.2f}ms")
print(f" 生成されたスパイク: {result['n_spikes_generated']}")

if result['is_fraud']:
 print(f"\nアラート: 手動分析のために取引がブロックされました！")

## 7. パフォーマンス分析

システムのレイテンシとスループットを評価します。

In [ ]:
# レイテンシベンチマーク
print("=== レイテンシベンチマーク ===")
n_samples = min(100, len(test_data))
print(f"{n_samples} 取引をテストしています...\n")

latencies = []
sample_txns = test_data.sample(n=n_samples)

for _, row in tqdm(sample_txns.iterrows(), total=n_samples, desc="ベンチマーク", unit="txn"):
 start = time.time()
 result = pipeline.predict(row.to_dict())
 latency = (time.time() - start) * 1000 # ms
 latencies.append(latency)

latencies = np.array(latencies)

print(f"\nレイテンシ統計:")
print(f" 平均: {latencies.mean():.2f}ms")
print(f" 中央値: {np.median(latencies):.2f}ms")
print(f" 最小: {latencies.min():.2f}ms")
print(f" 最大: {latencies.max():.2f}ms")
print(f" P95: {np.percentile(latencies, 95):.2f}ms")
print(f" P99: {np.percentile(latencies, 99):.2f}ms")

throughput = 1000 / latencies.mean() # 取引/秒
print(f"\n推定スループット: {throughput:.0f} 取引/秒")

# 分布を視覚化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(latencies, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(latencies.mean(), color='red', linestyle='--', label=f'平均: {latencies.mean():.2f}ms')
axes[0].axvline(np.percentile(latencies, 95), color='orange', linestyle='--', label=f'P95: {np.percentile(latencies, 95):.2f}ms')
axes[0].set_xlabel('レイテンシ（ms）')
axes[0].set_ylabel('頻度')
axes[0].set_title('レイテンシ分布')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].boxplot(latencies, vert=True)
axes[1].set_ylabel('レイテンシ（ms）')
axes[1].set_title('レイテンシ箱ひげ図')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 8. 結論

### ニューロモーフィックアプローチの利点

1. **超低レイテンシ**: 約10msで検出
2. **ネイティブ時間的処理**: シーケンスパターンを自然に捕捉
3. **エネルギー効率**: エッジデバイスでの展開に最適
4. **生物学的学習**: STDPにより継続的な適応が可能

### 銀行とフィンテックでの応用

- POSでのリアルタイム不正検出
- Pix/TED/DOC取引の保護
- デジタルウォレットの監視
- モバイルバンキングでの行動分析

### 次のステップ

- ニューロモーフィックハードウェアでの展開（Intel Loihi、IBM TrueNorth）
- API経由でのレガシーシステムとの統合
- 説明可能性（SNNのためのSHAP）
- 機関間の連合学習

---

**著者:** Mauro Risonho de Paula Assumpção
**プロジェクト:** 銀行サイバーセキュリティのためのニューロモーフィックコンピューティング